In [1]:
# Import necessary modules/libraries
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Import custom utility functions
from utils import p, expected, sampl_fair_opt_step, opt_step, fair_opt_step

In [6]:
# Set experiment parameters
np.random.seed(1)

# Distribution parameters
mean_a, std_a = 0.5, 1
mean_b, std_b = 0.0, 1

# Domain of alpha values to test
alpha_min, alpha_max = 0.01, 0.7
alphas = np.linspace(alpha_min, alpha_max, num=100)

# Sample size of each distribution
n = 10

# Generate samples from normal distributions
a = np.random.normal(mean_a, std_a, n)
b = np.random.normal(mean_b, std_b, n)
a.sort(), b.sort()
w_a = len(a) / (len(a) + len(b))
w_b = 1 - w_a

#Step Parameters

# U+ / U- > C+ / C-
u_plus = 1
u_minus = -1.1
c_plus = 1
c_minus = -1
# -0.4056
# -0.40551

In [ ]:
# Assumptions
'''
print(f' Assumption 1: {p.__doc__} ')
print(f' Assumption 2: \n U+ / U- > C+ / C- is {(u_plus / u_minus) > (c_plus / c_minus)}' )
'''

In [ ]:
# Single step optimization

x_alphas = []
y_mean_A, y_mean_B = [], []
y_thresh_A, y_thresh_B = [], []
y_util = []
y_pof = [] # Price of fairness


y_opt_util = []

B_temp = None

opt_util_A, opt_util_B = opt_step(a, u_plus, u_minus, c_plus, c_minus)[0], opt_step(b, u_plus, u_minus, c_plus, c_minus)[0]
opt_util_A, opt_util_B  = expected(opt_util_A, 1, -1.1), expected(opt_util_B, 1, -1.1)
opt_util_A, opt_util_B = np.sum(opt_util_A), np.sum(opt_util_B)
y_opt = w_a * opt_util_A + w_b * opt_util_B

for alpha in tqdm(alphas):
    results1 = sampl_fair_opt_step(a, b, u_plus, u_minus, c_plus, c_minus, alpha)
    results = fair_opt_step(a, b, u_plus, u_minus, c_plus, c_minus, alpha)
    if results != results1:
        print('error')
    thresh_A, thresh_B, max_util, (A, B) = results
    if np.abs(A - B) > alpha:
        continue
    else:
        x_alphas.append(alpha)
        y_mean_A.append(A)
        y_mean_B.append(B)
        y_thresh_A.append(thresh_A)
        y_thresh_B.append(thresh_B)
        y_util.append(max_util)
        pof = 1 - (max_util/y_opt)
        y_pof.append(pof)

In [ ]:
def plot_graphs():
    
    # Traces
    thresholds_A = go.Scatter(x=x_alphas, y=y_thresh_A, mode='markers', name="Fair Threshold (A)")
    thresholds_B = go.Scatter(x=x_alphas, y=y_thresh_B, mode='markers', name="Fair Threshold (B)") 
    utilities = go.Scatter(x=x_alphas, y=y_util, mode='markers', name='Utility', yaxis='y2')

    # Create figure
    fig = go.Figure(data=[thresholds_A, thresholds_B, utilities])

    # Add toggle buttons
    fig.update_layout(
        height = 600,
        title='Single Step Thresholding Policy',
        xaxis=dict(title="Alpha"),
        yaxis=dict(title="Threshold"),
        yaxis2=dict(
            title="Utility",
            overlaying="y",
            side="right"
        ),
        legend=dict(
            x=0,          # Right edge of the plotting area
            #y=1,          # Top of the plotting area
            xanchor='right',   # Legend's left edge aligns at x=1
            yanchor='top'     # Legend's top edge aligns at y=1
        ),
        showlegend=True,
    )

    fig.show()
plot_graphs()

In [ ]:
# Experiment 0: Check if vectorized algorithm is the same as the original
from experiment_0 import experiment_0
print( (x_alphas, y_thresh_A, y_thresh_B) == experiment_0(a, b, u_plus, u_minus, c_plus, c_minus, w_a, w_b, alphas, a, b) )

In [ ]:
# Experiment 2
from experiment_2 import experiment_2
test_alpha = 0.52
ex2a, ex2b = experiment_2(a, b, u_plus, u_minus, c_plus, c_minus, test_alpha, w_a, w_b, thresh_B, a, alphas)
ex2a.update_layout(
    xaxis_title ='Threshold A (using samples)'
)
ex2a.show()
#ex2b.show()

trace1 = ex2a.data[1]
trace2 = ex2a.data[2]
mean_diffs = dict(zip(trace1.x, trace1.y))
utilities = dict(zip(trace2.x, trace2.y))

y_thresh_A = np.array(y_thresh_A)
y_thresh_B = np.array(y_thresh_B)
x_alphas = np.array(x_alphas)
indices = np.where(y_thresh_A < y_thresh_B)[0]
test_alphas  = x_alphas[indices]
print(test_alphas)
#test_alphas = np.arange(0.54, 0.63, 0.01)
results = []

for test_alpha in test_alphas:
    # Filter mean_diffs under current threshold
    filtered = {k: v for k, v in mean_diffs.items() if v < test_alpha}

    # Find before and after keys
    before_keys = [k for k in filtered if k < 0]
    after_keys = [k for k in filtered if k > 0]

    before = max(before_keys) if before_keys else None
    after = min(after_keys) if after_keys else None

    if before is not None and after is not None:
        # Prepare data
        utility_before = utilities[before]
        utility_after = utilities[after]
        utility_diff = np.abs(utility_after - utility_before)

        # Build vertical block
        block = pd.DataFrame([
            {
                'Test Alpha': round(test_alpha, 3),
                'Side': 'Negative',
                'Threshold': before,
                'Mean Difference': mean_diffs[before],
                'Utility': utility_before
            },
            {
                'Test Alpha': round(test_alpha, 3),
                'Side': 'Positive',
                'Threshold': after,
                'Mean Difference': mean_diffs[after],
                'Utility': utility_after
            },
            {
                'Test Alpha': round(test_alpha, 3),
                'Side': 'Δ Utility',
                'Threshold': '',
                'Mean Difference': '',
                'Utility': utility_diff
            }
        ])

        print(block.to_string(index=False))
        print("--------")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def test(a, b, u_plus, u_minus, c_plus, c_minus, w_a, w_b):
    a = np.asarray(a)
    b = np.asarray(b)

    delta_A = expected(a, c_plus, c_minus)
    delta_B = expected(b, c_plus, c_minus)

    N = a.shape[0]
    A_matrix = np.repeat(a, N).reshape((N, N))
    B_matrix = np.repeat(b, N).reshape((N, N))
    delta_A_matrix = np.repeat(delta_A, N).reshape((N, N))
    delta_B_matrix = np.repeat(delta_B, N).reshape((N, N))

    A_matrix = np.where(A_matrix > A_matrix.T, A_matrix + delta_A_matrix, A_matrix)
    B_matrix = np.where(B_matrix > B_matrix.T, B_matrix + delta_B_matrix, B_matrix)

    columns = [(A_matrix[:, [i]], B_matrix[:, [i]]) for i in range(N)]
    final = []
    mean_diffs = []
    utilities = []

    for col_A_vec, col_B_vec in tqdm(columns):
        n = col_A_vec.shape[0]

        col_A_vec, col_B_vec = col_A_vec.T, col_B_vec.T

        delta_col_A = expected(col_A_vec, c_plus, c_minus)
        delta_col_B = expected(col_B_vec, c_plus, c_minus)

        col_A_matrix = np.repeat(col_A_vec, n).reshape((n, n))
        col_B_matrix = np.repeat(col_B_vec, n).reshape((n, n))

        delta_col_A_matrix = np.repeat(delta_col_A, n).reshape((n, n))
        delta_col_B_matrix = np.repeat(delta_col_B, n).reshape((n, n))

        col_A_matrix = np.where(col_A_matrix > col_A_matrix.T, col_A_matrix + delta_col_A_matrix, col_A_matrix)
        col_B_matrix = np.where(col_B_matrix > col_B_matrix.T, col_B_matrix + delta_col_B_matrix, col_B_matrix)

        final.append((col_A_matrix, col_B_matrix))

        # Compute utility from one matrix (e.g., col_A_matrix)
        util = w_a * expected(col_A_vec, u_plus, u_minus) + w_b * expected(col_B_vec, u_plus, u_minus)
        util = np.sum(util)
        utilities.append(util)

        # Compute mean absolute difference between vectors
        diff = np.mean(np.abs(col_A_vec - col_B_vec))
        mean_diffs.append(diff)

    # === Plot ===
    # === Plot Heatmap ===
    import matplotlib.pyplot as plt

    heatmap_data = utilities
    print(heatmap_data.shape)

    plt.figure(figsize=(7, 6))
    im = plt.imshow(heatmap_data, cmap='viridis', origin='lower', aspect='auto')
    plt.colorbar(im, label='Utility')
    plt.title("Utility Heatmap")
    plt.xlabel("Column Index")
    plt.ylabel("Row Index")
    plt.tight_layout()
    plt.show()

    return final
test(a, b, u_plus, u_minus, c_plus, c_minus, w_a, w_b)


In [38]:
# Vectorized double step
from utils import change

def vectorize_double_step(a, b, u_plus, u_minus, c_plus, c_minus, alphas, w_a, w_b):
    
    a = np.asarray(a)
    b = np.asarray(b)

    _, _, _, _,  A_matrix, B_matrix = change(a, b, c_plus, c_minus, u_plus, u_minus)
    A_matrix = A_matrix.T
    B_matrix = B_matrix.T

    mat_utils = []
    mat_diffs = []
    util = []

    # Iterate through each row of A_matrix and B_matrix
    for row_a, row_b in tqdm(zip(A_matrix, B_matrix)):
        mean_A, mean_B, util_A, util_B, _, _ = change(row_a, row_b, c_plus, c_minus, u_plus, u_minus)
        
        fairness_diff = np.abs(mean_A - mean_B)
        total_util = w_a * util_A + w_b * util_B.T


        mat_utils.append(total_util)
        mat_diffs.append(fairness_diff)
    
    utility = np.stack(mat_utils, axis=0) 
    mean_diff = np.stack(mat_diffs, axis=0)

    for alpha in alphas:

        temp = np.where(mean_diff <= alpha, utility, -np.inf)
        max_util = np.max(temp)
        '''
        # Build a list of masked total utilities satisfying the fairness constraint
        masked_utilities = []
        for total_util, fairness_diff in post_two_steps:
            # Apply fairness constraint: mask out unfair values
            masked = np.where(fairness_diff <= alpha, total_util, -np.inf)
            masked_utilities.append(masked)

        # Convert to 3D array: shape = (num_pairs, n, m)
        stacked_utils = np.array(masked_utilities)

        # Compute max across all utilities that met fairness constraint
        max_util = np.max(stacked_utils)

        # If all values are -inf (i.e. no valid entries), replace with np.nan
        if not np.isfinite(max_util):
            max_util = -np.inf
        '''
        util.append(max_util)

    plot1 = go.Scatter(x=alphas, y=util, mode='markers')
    fig = go.Figure(plot1)
    fig.update_layout(
        title='Alphas vs Utility (Dual Step, n^2 complexity)',
        xaxis_title='Alpha',
        yaxis_title='Utility'
        )
    fig.show()
    return np.array(alphas), np.array(util)

In [39]:
# Brute force double step
def double_step(a, b, u_plus, u_minus, c_plus, c_minus, alphas, w_a, w_b):
    
    delta_A1 = expected(a, c_plus, c_minus)
    delta_B1 = expected(b, c_plus, c_minus)

    utilities = []
    mean_diffs = []
    thresholds = []

    for alpha in tqdm(alphas):
        max_util = -np.inf
        fair = -np.inf 
        for i in a:
            for j in b:   
                # Apply the first step
                A = np.where(a > i, a + delta_A1, a)
                B = np.where(b > j, b + delta_B1, b)
                
                # Start the second step
                delta_A2 = expected(A, c_plus, c_minus)
                delta_B2 = expected(B, c_plus, c_minus)
                
                for k in A:
                    for l in B:
                        A2 = np.where(A > k, A + delta_A2, A)
                        B2 = np.where(B > l, B + delta_B2, B)
                        
                        util = w_a * np.sum(expected(A2, u_plus, u_minus)) + w_b * np.sum(expected(B2, u_plus, u_minus))
                        diff = np.abs(np.mean(A2)- np.mean(B2))
                        
                        if util > max_util and diff <= alpha:
                            max_util = util
                            fair = diff
                        
        utilities.append(max_util)
        mean_diffs.append(fair)
        thresholds.append((i, j, k, l))
    utility = go.Scatter(x=alphas, y=utilities, mode='markers', name='Utility')
    mean_diff = go.Scatter(x=alphas, y=mean_diffs, mode='markers')
    fig = go.Figure(data=[utility, mean_diff])
    fig.update_layout(
        title='Alpha vs Utility (Dual Step, n^4 Complexity',
        xaxis_title='Alpha',
        yaxis_title='Utility'
            )
    fig.show()

    return np.array(alphas), np.array(utilities), np.array(mean_diffs)
#double_step(a, b, u_plus, u_minus, c_plus, c_minus, alphas, w_a, w_b)



In [41]:
alphas2, utils2 = vectorize_double_step(a, b, u_plus, u_minus, c_plus, c_minus, alphas, w_a, w_b)
alphas1, utils1, _ = double_step(a, b, u_plus, u_minus, c_plus, c_minus, alphas, w_a, w_b)

10it [00:00, 5806.87it/s]


100%|██████████| 100/100 [00:38<00:00,  2.57it/s]


In [37]:
print(np.allclose(utils1, utils2, equal_nan=True))
print([(i, j) for i, j in zip(utils1, utils2) if not np.isclose(i, j, equal_nan=True)])
print(utils1)
print('\n')
print(utils2)

False
[(np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.39619295398545407), np.float64(-inf)), (np.float64(0.44944800394220197), np.float64(-inf)), (np.float64(0.44944800394220197), np.float64(-inf)), (np.float64(0.44944800394220197), np.float64(-inf)), (np.float64(0.44944800394220197), np.float64(-inf)), (np.float64(0.44944800394220197), np.float64(-inf)), (np.float64(0.44944800394220197), np.float64(-inf)), (np.float64(0.44944800394220197), np.fl